<a href="https://colab.research.google.com/github/hollman/boxrecv1/blob/main/Python_Scraper_for_BOXREC_Data_Population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
import psycopg2
from google.cloud import firestore
from google.cloud.sql.connector import Connector, IPTypes

# /// USER ACTION REQUIRED ///
# -----------------------------------------------------------------------------
# 1. Update these variables with your specific project details.
# 2. Make sure the service account key file is accessible at the path you provide.
# -----------------------------------------------------------------------------
GCP_PROJECT_ID = "YOUR_GCP_PROJECT_ID"
GCP_REGION = "YOUR_GCP_REGION" # e.g., "us-east1"
INSTANCE_CONNECTION_NAME = "YOUR_INSTANCE_CONNECTION_NAME" # Get this from the Cloud SQL instance page
DB_USER = "postgres"
DB_PASS = "YOUR_DB_PASSWORD" # The password you set during instance creation
DB_NAME = "postgres" # Default database name
SERVICE_ACCOUNT_KEY_FILE = "PATH_TO_YOUR_KEY_FILE.json"

# --- Globals ---
db = None
firestore_client = None

def get_db_connection():
    """Initializes a connection pool for a Cloud SQL instance of Postgres."""
    global db
    if db:
        return db

    # Initialize Google Cloud SQL Connector
    connector = Connector()

    def get_conn() -> psycopg2.extensions.connection:
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pg8000",
            user=DB_USER,
            password=DB_PASS,
            db=DB_NAME,
            ip_type=IPTypes.PUBLIC,
        )
        return conn

    # Create connection pool
    try:
        db = get_conn()
        print("Successfully connected to Cloud SQL database.")
        return db
    except Exception as e:
        print(f"Error connecting to Cloud SQL: {e}")
        raise

def get_firestore_client():
    """Initializes the Firestore client."""
    global firestore_client
    if firestore_client:
        return firestore_client

    try:
        # Set the environment variable for authentication
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_KEY_FILE
        firestore_client = firestore.Client(project=GCP_PROJECT_ID)
        print("Successfully connected to Firestore.")
        return firestore_client
    except Exception as e:
        print(f"Error connecting to Firestore: {e}")
        raise

def setup_database_schema(conn):
    """Creates the necessary tables in the PostgreSQL database if they don't exist."""

    # Note: For simplicity, Venues and Officials are not fully fleshed out here,
    # but the tables are created for future use.

    table_creation_queries = [
        """
        CREATE TABLE IF NOT EXISTS Boxers (
            boxer_id INT PRIMARY KEY, name VARCHAR(255) NOT NULL, alias VARCHAR(255),
            nickname VARCHAR(255), sex VARCHAR(10), status VARCHAR(20), age INT,
            stance VARCHAR(20), height_cm INT, reach_cm INT, nationality VARCHAR(100),
            birth_date DATE, birth_place TEXT, residence TEXT, global_rating NUMERIC(8, 2),
            division_rating NUMERIC(8, 2), division_rank INT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Venues (
            venue_id INT PRIMARY KEY, name VARCHAR(255) NOT NULL, city VARCHAR(100),
            country VARCHAR(100), address TEXT
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Events (
            event_id INT PRIMARY KEY, event_name VARCHAR(255), event_date DATE,
            status VARCHAR(50), country VARCHAR(100), city VARCHAR(100),
            venue_id INT -- REFERENCES Venues(venue_id) -- FK can be added later
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Officials (
            official_id INT PRIMARY KEY, name VARCHAR(255) NOT NULL, role VARCHAR(50),
            country VARCHAR(100), birth_date DATE
        );
        """,
        """
        CREATE TABLE IF NOT EXISTS Bouts (
            bout_id INT PRIMARY KEY, event_id INT, -- REFERENCES Events(event_id),
            bout_date DATE, division VARCHAR(100), boxer1_id INT, -- REFERENCES Boxers(boxer_id),
            boxer2_id INT, -- REFERENCES Boxers(boxer_id),
            result VARCHAR(50), method VARCHAR(100), rounds_fought INT,
            rounds_scheduled INT, referee_id INT -- REFERENCES Officials(official_id)
        );
        """
    ]

    try:
        with conn.cursor() as cursor:
            print("Setting up database schema...")
            for query in table_creation_queries:
                cursor.execute(query)
            conn.commit()
            print("Schema setup complete.")
    except Exception as e:
        print(f"Error setting up schema: {e}")
        conn.rollback()
        raise

def scrape_and_process_boxer(boxer_id, conn):
    """Scrapes a single boxer's page and inserts their data into the database."""
    url = f"https://boxrec.com/en/boxer/{boxer_id}"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Raises an exception for bad status codes
        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Scrape Boxer Profile Data ---
        profile_table = soup.find('table', class_='profileTable')
        if not profile_table:
            print(f"Boxer ID {boxer_id}: Profile table not found. Skipping.")
            return

        name = soup.find('h1').text.strip() if soup.find('h1') else 'N/A'

        # This is a simplified extraction. BOXREC's HTML can be complex.
        # We look for labels and get their sibling data.
        data = {'boxer_id': boxer_id, 'name': name}

        # A helper to find data by its label
        def get_value_by_label(label_text):
            label_element = profile_table.find(lambda tag: tag.name == 'td' and label_text in tag.text)
            if label_element and label_element.find_next_sibling('td'):
                return label_element.find_next_sibling('td').text.strip()
            return None

        data['status'] = get_value_by_label('status')

        # ... add more scraping logic for other fields like stance, height, etc.

        # --- Insert Boxer into Database ---
        with conn.cursor() as cursor:
            cursor.execute(
                """
                INSERT INTO Boxers (boxer_id, name, status)
                VALUES (%s, %s, %s)
                ON CONFLICT (boxer_id) DO NOTHING;
                """,
                (data['boxer_id'], data['name'], data['status'])
            )
        print(f"Processed Boxer ID {boxer_id}: {name}")

        # --- Scrape and Insert Bouts ---
        bouts_table = soup.find('table', class_='dataTable')
        if bouts_table and bouts_table.find('tbody'):
            for row in bouts_table.find('tbody').find_all('tr'):
                cols = row.find_all('td')
                if len(cols) > 10: # A basic check for a valid bout row
                    bout_date = cols[0].text.strip()
                    opponent_link = cols[4].find('a')
                    opponent_id = opponent_link['href'].split('/')[-1] if opponent_link else None

                    bout_id_link = cols[-1].find('a')
                    bout_id = bout_id_link['href'].split('/')[-1] if bout_id_link else None

                    if bout_id and opponent_id:
                         with conn.cursor() as cursor:
                            cursor.execute(
                                """
                                INSERT INTO Bouts (bout_id, bout_date, boxer1_id, boxer2_id)
                                VALUES (%s, %s, %s, %s)
                                ON CONFLICT (bout_id) DO NOTHING;
                                """,
                                (bout_id, bout_date, boxer_id, opponent_id)
                            )
        conn.commit()

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Boxer ID {boxer_id}: Not found (404). Skipping.")
        else:
            print(f"HTTP error for Boxer ID {boxer_id}: {e}")
    except Exception as e:
        print(f"An error occurred while processing Boxer ID {boxer_id}: {e}")
        conn.rollback()


def main():
    """Main function to run the scraper."""
    print("Starting BOXREC Data Scraper...")

    # Establish connections
    sql_conn = get_db_connection()
    #firestore_client = get_firestore_client() # Uncomment if you plan to scrape News

    if not sql_conn:
        print("Could not establish database connection. Exiting.")
        return

    # Create tables if they don't exist
    setup_database_schema(sql_conn)

    # --- Main Scraping Loop ---
    # We will iterate through a range of boxer IDs.
    # Adjust the range as needed. BOXREC has IDs well into the millions.
    # We'll start with a small, recent range to test.
    start_id = 850000
    end_id = 851000 # Scrape 1000 boxers

    print(f"Starting to scrape boxer IDs from {start_id} to {end_id}...")

    for boxer_id in range(start_id, end_id + 1):
        scrape_and_process_boxer(boxer_id, sql_conn)

        # Be a good internet citizen and avoid getting blocked.
        time.sleep(1) # Wait 1 second between requests

    print("Scraping finished.")

    # Close resources
    sql_conn.close()
    connector = Connector()
    connector.close()
    print("Database connection closed.")


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'google.cloud.sql'